# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Wen cova #

#### Import The Necesary Libreries ####

## Question Number 1 ##

### Importing  Packages ###

In [3]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

Packages installed.


In [4]:
pip install BeautifulSoup4

     |████████████████████████████████| 122kB 6.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Getting the Data ###

In [6]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 
My_table = soup.find('table',{'class':'wikitable sortable'})


print(soup.prettify()) 
print('Page Scrapped.')

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"652df336-7a6c-4f10-8854-52123f8436fb","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

### Looking the table ###

In [7]:
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [8]:
print(My_table.tr.text)


Postal Code

Borough

Neighborhood



In [9]:
headers="Postcode,Borough,Neighbourhood"

### Getting all values and separating with , ###

In [10]:
table1=""
for tr in My_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
print(table1)

M1A
,Not assigned
,Not assigned
M2A
,Not assigned
,Not assigned
M3A
,North York
,Parkwoods
M4A
,North York
,Victoria Village
M5A
,Downtown Toronto
,Regent Park, Harbourfront
M6A
,North York
,Lawrence Manor, Lawrence Heights
M7A
,Downtown Toronto
,Queen's Park, Ontario Provincial Government
M8A
,Not assigned
,Not assigned
M9A
,Etobicoke
,Islington Avenue, Humber Valley Village
M1B
,Scarborough
,Malvern, Rouge
M2B
,Not assigned
,Not assigned
M3B
,North York
,Don Mills
M4B
,East York
,Parkview Hill, Woodbine Gardens
M5B
,Downtown Toronto
,Garden District, Ryerson
M6B
,North York
,Glencairn
M7B
,Not assigned
,Not assigned
M8B
,Not assigned
,Not assigned
M9B
,Etobicoke
,West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
M1C
,Scarborough
,Rouge Hill, Port Union, Highland Creek
M2C
,Not assigned
,Not assigned
M3C
,North York
,Don Mills
M4C
,East York
,Woodbine Heights
M5C
,Downtown Toronto
,St. James Town
M6C
,York
,Humewood-Cedarvale
M7C
,Not assigned
,Not assigned
M8C
,N

### Converting into dataframe and assigning column names ###

In [11]:


postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')



Data Collected.


In [12]:


# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors



Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood, Latitude, Longitude]
Index: []

In [13]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

PostalCode             Borough  \
0        M1A\n      Not assigned\n   
1        M2A\n      Not assigned\n   
2        M3A\n        North York\n   
3        M4A\n        North York\n   
4        M5A\n  Downtown Toronto\n   
..         ...                 ...   
174      M4Z\n      Not assigned\n   
175      M5Z\n      Not assigned\n   
176      M6Z\n      Not assigned\n   
177      M7Z\n      Not assigned\n   
178      M8Z\n         Etobicoke\n   

                                          Neighborhood   Latitude  Longitude  
0                                       Not assigned\n  43.648690 -79.385440  
1                                       Not assigned\n  43.648690 -79.385440  
2                                          Parkwoods\n  43.752935 -79.335641  
3                                   Victoria Village\n  43.728102 -79.311890  
4                          Regent Park, Harbourfront\n  43.650964 -79.353041  
..                                                 ...        ...        ...  
174                                     Not assigned\n  43.648690 -79.385440  
175                                     Not assigned\n  43.648690 -79.385440  
176                                     Not assigned\n  43.648690 -79.385440  
177                                     Not assigned\n  43.648690 -79.385440  
178  Mimico NW, The Queensway West, South of Bloor,...  43.625490 -79.526000  

[179 rows x 5 columns]

In [14]:
neighbors.shape

(179, 5)

In [15]:
neighbors.head(10)

PostalCode             Borough  \
0      M1A\n      Not assigned\n   
1      M2A\n      Not assigned\n   
2      M3A\n        North York\n   
3      M4A\n        North York\n   
4      M5A\n  Downtown Toronto\n   
5      M6A\n        North York\n   
6      M7A\n  Downtown Toronto\n   
7      M8A\n      Not assigned\n   
8      M9A\n         Etobicoke\n   
9      M1B\n       Scarborough\n   

                                    Neighborhood   Latitude  Longitude  
0                                 Not assigned\n  43.648690 -79.385440  
1                                 Not assigned\n  43.648690 -79.385440  
2                                    Parkwoods\n  43.752935 -79.335641  
3                             Victoria Village\n  43.728102 -79.311890  
4                    Regent Park, Harbourfront\n  43.650964 -79.353041  
5             Lawrence Manor, Lawrence Heights\n  43.723265 -79.451211  
6  Queen's Park, Ontario Provincial Government\n  43.661790 -79.389390  
7                                 Not assigned\n  43.648690 -79.385440  
8      Islington Avenue, Humber Valley Village\n  43.667481 -79.528953  
9                               Malvern, Rouge\n  43.808626 -79.189913

### removing /n ###

In [16]:
neighbors = neighbors.replace('\n','', regex=True)

In [17]:
neighbors.head(10)

PostalCode           Borough                                 Neighborhood  \
0        M1A      Not assigned                                 Not assigned   
1        M2A      Not assigned                                 Not assigned   
2        M3A        North York                                    Parkwoods   
3        M4A        North York                             Victoria Village   
4        M5A  Downtown Toronto                    Regent Park, Harbourfront   
5        M6A        North York             Lawrence Manor, Lawrence Heights   
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
7        M8A      Not assigned                                 Not assigned   
8        M9A         Etobicoke      Islington Avenue, Humber Valley Village   
9        M1B       Scarborough                               Malvern, Rouge   

    Latitude  Longitude  
0  43.648690 -79.385440  
1  43.648690 -79.385440  
2  43.752935 -79.335641  
3  43.728102 -79.311890  
4  43.650964 -79.353041  
5  43.723265 -79.451211  
6  43.661790 -79.389390  
7  43.648690 -79.385440  
8  43.667481 -79.528953  
9  43.808626 -79.189913

### Remove Unasigned Values ###

In [18]:

# Get names of indexes for which column Borough has value "Not assigned"
indexNames = neighbors[ neighbors['Borough'] =='Not assigned'].index

# Delete these row indexes from dataFrame
neighbors.drop(indexNames , inplace=True)



In [19]:
neighbors.head(10)

PostalCode           Borough                                 Neighborhood  \
2         M3A        North York                                    Parkwoods   
3         M4A        North York                             Victoria Village   
4         M5A  Downtown Toronto                    Regent Park, Harbourfront   
5         M6A        North York             Lawrence Manor, Lawrence Heights   
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
8         M9A         Etobicoke      Islington Avenue, Humber Valley Village   
9         M1B       Scarborough                               Malvern, Rouge   
11        M3B        North York                                    Don Mills   
12        M4B         East York              Parkview Hill, Woodbine Gardens   
13        M5B  Downtown Toronto                     Garden District, Ryerson   

     Latitude  Longitude  
2   43.752935 -79.335641  
3   43.728102 -79.311890  
4   43.650964 -79.353041  
5   43.723265 -79.451211  
6   43.661790 -79.389390  
8   43.667481 -79.528953  
9   43.808626 -79.189913  
11  43.748900 -79.357220  
12  43.707193 -79.311529  
13  43.657491 -79.377529

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough ###

In [20]:

neighbors.loc[neighbors['Neighborhood'] =='Not assigned' , 'Neighborhood'] = neighbors['Borough']
neighbors.head(10)



PostalCode           Borough                                 Neighborhood  \
2         M3A        North York                                    Parkwoods   
3         M4A        North York                             Victoria Village   
4         M5A  Downtown Toronto                    Regent Park, Harbourfront   
5         M6A        North York             Lawrence Manor, Lawrence Heights   
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
8         M9A         Etobicoke      Islington Avenue, Humber Valley Village   
9         M1B       Scarborough                               Malvern, Rouge   
11        M3B        North York                                    Don Mills   
12        M4B         East York              Parkview Hill, Woodbine Gardens   
13        M5B  Downtown Toronto                     Garden District, Ryerson   

     Latitude  Longitude  
2   43.752935 -79.335641  
3   43.728102 -79.311890  
4   43.650964 -79.353041  
5   43.723265 -79.451211  
6   43.661790 -79.389390  
8   43.667481 -79.528953  
9   43.808626 -79.189913  
11  43.748900 -79.357220  
12  43.707193 -79.311529  
13  43.657491 -79.377529

### rows will be same postalcode will combined into one row with the neighborhoods separated with a comma ##

In [21]:
result = neighbors.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join)

In [22]:
df_new=result.reset_index()
df_new.head(15)

PostalCode           Borough  \
0         M3A        North York   
1         M4A        North York   
2         M5A  Downtown Toronto   
3         M6A        North York   
4         M7A  Downtown Toronto   
5         M9A         Etobicoke   
6         M1B       Scarborough   
7         M3B        North York   
8         M4B         East York   
9         M5B  Downtown Toronto   
10        M6B        North York   
11        M9B         Etobicoke   
12        M1C       Scarborough   
13        M3C        North York   
14        M4C         East York   

                                         Neighborhood  
0                                           Parkwoods  
1                                    Victoria Village  
2                           Regent Park, Harbourfront  
3                    Lawrence Manor, Lawrence Heights  
4         Queen's Park, Ontario Provincial Government  
5             Islington Avenue, Humber Valley Village  
6                                      Malvern, Rouge  
7                                           Don Mills  
8                     Parkview Hill, Woodbine Gardens  
9                            Garden District, Ryerson  
10                                          Glencairn  
11  West Deane Park, Princess Gardens, Martin Grov...  
12             Rouge Hill, Port Union, Highland Creek  
13                                          Don Mills  
14                                   Woodbine Heights

### Finally use the .shape method to print the number of rows of your dataframe ###

In [23]:
df_new.shape

(103, 3)